In [24]:
import pandas as pd
import folium
from math import radians, sin, cos, sqrt, atan2
from branca.element import Element

# --- 1. 데이터 불러오기 및 전처리 ---
try:
    df = pd.read_csv('restaurants.csv', encoding='utf-8-sig')
    df.columns = df.columns.str.replace('\ufeff', '').str.strip()
    df['위도'] = pd.to_numeric(df['위도'], errors='coerce')
    df['경도'] = pd.to_numeric(df['경도'], errors='coerce')
except:
    df = pd.DataFrame(columns=['이름', '위도', '경도', '영업시간', '휴무', '평점', '대표메뉴', '종류', '전화번호'])

# --- 2. 색상 설정 ---
color_dict = {
    '한식': '#E94B3C',
    '분식': '#3867B4',
    '카페': '#9b59b6',
    '중식': '#F39C12',
    '일식': '#6C5CE7',
    '해산물': '#16a085'
}
center_lat, center_lng = 35.63110, 129.35475

# --- 3. 반경 내 여부 계산 함수 ---
def is_within_radius(lat1, lon1, lat2, lon2, radius_km=5):
    R = 6371
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c <= radius_km

# --- 4. 지도 생성 ---
m = folium.Map(location=[center_lat, center_lng], zoom_start=14)

# 내 위치 마커
folium.Marker(
    location=[center_lat, center_lng],
    icon=folium.Icon(color='red', icon='home', prefix='fa'),
    tooltip='내 위치 (호계고등학교 기준)'
).add_to(m)

# 반경 5km 원
folium.Circle(
    location=[center_lat, center_lng], radius=5000,
    color='blue', fill=False, weight=2, dash_array='5,10', tooltip='반경 5km'
).add_to(m)

# --- 5. 마커 저장용 딕셔너리 ---
markers_by_type = {key: [] for key in color_dict.keys()}

# --- 6. CSV 데이터 음식점 마커 생성 ---
for idx, row in df.iterrows():
    name = row['이름']
    lat, lng = row['위도'], row['경도']
    if pd.isna(lat) or pd.isna(lng):
        continue
    if not is_within_radius(center_lat, center_lng, lat, lng):
        continue

    food_type = row['종류'] if row['종류'] in color_dict else None
    if not food_type:
        continue

    open_time = str(row['영업시간']).replace('\n', '<br>')
    popup_html = f"""
    <div style='width:280px; font-size:14px; font-family:sans-serif; line-height:1.5;'>
        <h4 style='margin-bottom:4px; color:{color_dict[food_type]};'>{name}</h4>
        <p>종류: {food_type}</p>
        <p>대표메뉴: {row['대표메뉴']}</p>
        <p>평점: <span style='color:#FFA000;'>★</span> {row['평점']}</p>
        <p>영업시간:<br>{open_time}</p>
        <p>휴무: {row['휴무']}</p>
        <p>전화번호: {row.get('전화번호', '정보없음')}</p>
    </div>
    """
    marker = folium.CircleMarker(
        location=[lat, lng], radius=9,
        color=color_dict[food_type],
        fill=True, fill_color=color_dict[food_type],
        fill_opacity=0.85,
        popup=folium.Popup(popup_html, max_width=300),
        tooltip=name,
        # 클래스명으로 필터링에 사용
        **{'className': f'marker-{food_type}'}
    )
    markers_by_type[food_type].append(marker)
    marker.add_to(m)

# --- 7. 직접 추가 음식점 ---
extra_restaurants = [
    {
        '이름': '기린식당', '위도': 35.6318, '경도': 129.3543,
        '영업시간': '11:00 - 20:30 (수요일: 낮 운영)', '휴무': '없음',
        '평점': 4.5, '대표메뉴': '부타카쿠니 덮밥', '종류': '일식', '전화번호': '0507-1429-1136'
    },
    {
        '이름': '황금반점', '위도': 35.6330, '경도': 129.3550,
        '영업시간': '11:00 - 21:00', '휴무': '없음',
        '평점': 4.3, '대표메뉴': '짜장면, 탕수육', '종류': '중식', '전화번호': '052-123-4567'
    },
    {
        '이름': '바다향기', '위도': 35.6295, '경도': 129.3520,
        '영업시간': '10:00 - 22:00', '휴무': '화요일',
        '평점': 4.6, '대표메뉴': '조개찜', '종류': '해산물', '전화번호': '052-234-5678'
    },
    {
        '이름': '멋진카페', '위도': 35.6320, '경도': 129.3500,
        '영업시간': '08:00 - 20:00', '휴무': '없음',
        '평점': 4.1, '대표메뉴': '아메리카노', '종류': '카페', '전화번호': '052-345-6789'
    }
]

for rest in extra_restaurants:
    if is_within_radius(center_lat, center_lng, rest['위도'], rest['경도']):
        food_type = rest['종류']
        popup_html = f"""
        <div style='width:280px; font-size:14px; font-family:sans-serif; line-height:1.5;'>
            <h4 style='margin-bottom:4px; color:{color_dict.get(food_type, "#7f8c8d")};'>{rest['이름']}</h4>
            <p>종류: {food_type}</p>
            <p>대표메뉴: {rest['대표메뉴']}</p>
            <p>평점: <span style='color:#FFA000;'>★</span> {rest['평점']}</p>
            <p>영업시간:<br>{rest['영업시간']}</p>
            <p>휴무: {rest['휴무']}</p>
            <p>전화번호: {rest['전화번호']}</p>
        </div>
        """
        marker = folium.CircleMarker(
            location=[rest['위도'], rest['경도']], radius=9,
            color=color_dict.get(food_type, '#7f8c8d'),
            fill=True, fill_color=color_dict.get(food_type, '#7f8c8d'),
            fill_opacity=0.85,
            popup=folium.Popup(popup_html, max_width=300),
            tooltip=rest['이름'],
            **{'className': f'marker-{food_type}'}
        )
        markers_by_type[food_type].append(marker)
        marker.add_to(m)

# --- 8. 필터링 기능을 위한 JS ---
filter_script = """
<script>
function setVisibility(type) {
    const allMarkers = document.querySelectorAll('.leaflet-interactive');
    allMarkers.forEach(el => {
        if (type === '전체') {
            el.style.display = 'block';
        } else {
            if (el.classList.contains(`marker-${type}`)) {
                el.style.display = 'block';
            } else {
                el.style.display = 'none';
            }
        }
    });
}
</script>
"""

# --- 9. 필터 버튼 UI ---
filter_buttons = '''
<style>
#filter-box {
    position: fixed;
    top: 20px;
    right: 12px;
    width: 150px;
    background: #2d3436;
    padding: 12px 15px;
    border-radius: 12px;
    border: none;
    z-index: 9999;
    font-size: 14px;
    box-shadow: 0 4px 8px rgba(0,0,0,0.3);
    font-family: "Segoe UI", Tahoma, Geneva, Verdana, sans-serif;
}
#filter-box button {
    margin: 6px 0;
    width: 100%;
    padding: 8px 0;
    border: none;
    background: #00cec9;
    color: #fff;
    border-radius: 25px;
    cursor: pointer;
    font-weight: 600;
    font-size: 15px;
    transition: background-color 0.3s ease;
    box-shadow: 0 2px 6px rgba(0,0,0,0.15);
}
#filter-box button:hover {
    background: #81ecec;
    color: #2d3436;
}
#filter-box button:focus {
    outline: none;
    box-shadow: 0 0 8px 2px #00cec9;
}
</style>
<div id="filter-box">
  <button onclick="setVisibility('전체')" style="background:#0984e3;">전체 보기</button>
  <button onclick="setVisibility('한식')">한식</button>
  <button onclick="setVisibility('분식')">분식</button>
  <button onclick="setVisibility('카페')">카페</button>
  <button onclick="setVisibility('중식')">중식</button>
  <button onclick="setVisibility('일식')">일식</button>
  <button onclick="setVisibility('해산물')">해산물</button>
</div>
'''

# --- 10. 음식점 종류 범례 ---
legend_html = '''
<style>
#legend-toggle {
    position: fixed;
    top: 130px;
    left: 12px;
    background: #2d3436;
    color: #fff;
    padding: 10px 14px;
    border: none;
    border-radius: 8px;
    font-size: 15px;
    z-index: 9999;
    cursor: pointer;
    font-family: "Segoe UI", Tahoma, Geneva, Verdana, sans-serif;
}
#legend-box {
    position: fixed;
    top: 180px;
    left: 12px;
    width: 190px;
    padding: 14px;
    background: #fff;
    border: 2px solid #ccc;
    border-radius: 12px;
    z-index: 9999;
    font-size: 15px;
    display: none;
    line-height: 1.6;
    font-family: "Segoe UI", Tahoma, Geneva, Verdana, sans-serif;
}
#legend-box i {
    width: 16px;
    height: 16px;
    display: inline-block;
    border-radius: 50%;
    margin-right: 8px;
    vertical-align: middle;
}
</style>
<button id="legend-toggle">🍴 음식점 종류 보기</button>
<div id="legend-box">
  <b>🍴 음식점 종류</b><br><br>
  <i style="background:#E94B3C;"></i> 한식<br>
  <i style="background:#3867B4;"></i> 분식<br>
  <i style="background:#9b59b6;"></i> 카페<br>
  <i style="background:#F39C12;"></i> 중식<br>
  <i style="background:#6C5CE7;"></i> 일식<br>
  <i style="background:#16a085;"></i> 해산물<br>
</div>
<script>
const toggleBtn = document.getElementById('legend-toggle');
const legendBox = document.getElementById('legend-box');
toggleBtn.addEventListener('click', () => {
    if (legendBox.style.display === 'none' || legendBox.style.display === '') {
        legendBox.style.display = 'block';
        toggleBtn.innerText = '❌ 음식점 종류 닫기';
    } else {
        legendBox.style.display = 'none';
        toggleBtn.innerText = '🍴 음식점 종류 보기';
    }
});
</script>
'''

# --- 11. 반경 안내문 ---
radius_info_html = '''
<div id="radius-info" style="position: fixed; bottom: 16px; left: 12px; background: #ffffffcc; color: #2d3436;
padding: 10px 14px; font-size: 15px; border: 2px solid #ccc; border-radius: 10px; z-index: 9999;
max-width: 320px; box-shadow: 2px 2px 6px rgba(0,0,0,0.2); font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;">
📍 현재 지도에는 <b>호계고등학교 기준 반경 5km 이내</b> 음식점만 표시됩니다.
</div>
'''

# --- 12. 반응형 CSS 추가 ---
responsive_css = """
<style>
/* 모바일 및 작은 화면 대응 */
@media (max-width: 768px) {
    #filter-box {
        width: 130px !important;
        top: 10px !important;
        right: 8px !important;
        font-size: 13px !important;
        padding: 10px 12px !important;
    }
    #filter-box button {
        font-size: 13px !important;
        padding: 6px 0 !important;
        margin: 4px 0 !important;
    }
    #legend-toggle {
        top: 110px !important;
        left: 8px !important;
        font-size: 13px !important;
        padding: 8px 12px !important;
    }
    #legend-box {
        top: 150px !important;
        left: 8px !important;
        width: 160px !important;
        font-size: 13px !important;
        padding: 12px !important;
    }
    #radius-info {
        bottom: 12px !important;
        left: 8px !important;
        font-size: 13px !important;
        max-width: 280px !important;
        padding: 8px 12px !important;
    }
}
</style>
"""

# --- 13. HTML 요소 추가 ---
m.get_root().html.add_child(Element(filter_script))
m.get_root().html.add_child(Element(filter_buttons))
m.get_root().html.add_child(Element(legend_html))
m.get_root().html.add_child(Element(radius_info_html))
m.get_root().html.add_child(Element(responsive_css))

# --- 14. 지도 저장 ---
m.save('restaurant_map.html')
